In [37]:
import numpy as np
import pandas as pd
import pymongo
import os
import pickle
import uuid
from sklearn.manifold import TSNE

In [29]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [30]:
dataset_directory=os.path.join("..","api","dataset","v2")
def shortid(num):
    return [str(uuid.uuid4())[:8] for i in range(num)]
def label_encode(string):
    return string.replace("\\'",".").replace(" ","_")

In [52]:
def dataset_to_mongodb(dataset_name, dataset_type):
    collection_name = dataset_name + "_" + dataset_type

    # open file
    with open(os.path.join(dataset_directory, dataset_name, dataset_type + ".pickle"), 'rb') as f:
        X_train, y_train, feature_names, label_names = pickle.load(f)

    # assign ids
    index=shortid(X_train.shape[0])

    # insert feature data
    X_train=pd.DataFrame(X_train.todense(),columns=[feature_names[x][0] for x in range(X_train.shape[1])],index=index)
    X_train["_id"]=index
    myclient[collection_name]["features"].insert_many(X_train.to_dict('records'))

    # insert label data
    y_train=pd.DataFrame(y_train.todense(),columns=[label_encode(label_names[x][0]) for x in range(y_train.shape[1])],index=index)
    y_train["_id"]=index
    myclient[collection_name]["labels"].insert_many(y_train.to_dict('records'))

    # insert feature tsne
    t_sne = TSNE()
    t_sne.fit(X_train.drop("_id",axis=1))
    t_sne_df = pd.DataFrame(t_sne.embedding_, columns=["v1","v2"], index=index)
    t_sne_df["_id"]=index
    myclient[collection_name]["tsne_features"].insert_many(t_sne_df.to_dict('records'))

    # insert label tsne
    t_sne = TSNE()
    t_sne.fit(y_train.drop("_id",axis=1))
    t_sne_df = pd.DataFrame(t_sne.embedding_, columns=["v1","v2"], index=index)
    t_sne_df["_id"]=index
    myclient[collection_name]["tsne_labels"].insert_many(t_sne_df.to_dict('records'))

In [53]:
dataset_to_mongodb("birds","train")
dataset_to_mongodb("birds","test")
dataset_to_mongodb("emotions","train")
dataset_to_mongodb("emotions","test")
dataset_to_mongodb("delicous","test")

/Users/phlin/Library/Python/3.8/lib/python/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/phlin/Library/Python/3.8/lib/python/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/phlin/Library/Python/3.8/lib/python/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/phlin/Library/Python/3.8/lib/python/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/phlin/Library/Python/3.8/lib/python/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/phli

FileNotFoundError: [Errno 2] No such file or directory: '../api/dataset/v2/delicous/test.pickle'